In [5]:
!pip freeze

absl-py==1.2.0
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.4
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.8
certifi==2022.6.15
cffi==1.15.1
cftime==1.6.1
chardet==3.0.4
charset-normalizer==2.1.0
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.4
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.1
cycler==0.11.0
cymem==2.0.6
Cython==0.29.32
daft==0.0.4
dask==2022.2.0
datascience==0.17.5
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
deprecat==2.1.1
descartes==1.1.0
dill=

In [8]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [9]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.24.jar

--2022-08-22 05:54:55--  https://jdbc.postgresql.org/download/postgresql-42.2.24.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1006549 (983K) [application/java-archive]
Saving to: ‘postgresql-42.2.24.jar’

postgresql-42.2.24. 100%[===================>] 982.96K  6.33MB/s    in 0.2s    

2022-08-22 05:54:56 (6.33 MB/s) - ‘postgresql-42.2.24.jar’ saved [1006549/1006549]



In [10]:
# Adds the driver to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.24.jar").getOrCreate()

In [11]:
# Extract
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://nasa-bucket.s3.amazonaws.com/nm_value_df.csv"
spark.sparkContext.addFile(url)
nm_value_df = spark.read.csv(SparkFiles.get("nm_value_df.csv"), sep=",", header=True, inferSchema=True)

In [12]:
# Extract
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://nasa-bucket.s3.amazonaws.com/planet_prediction_df.csv"
spark.sparkContext.addFile(url)
planet_prediction_df = spark.read.csv(SparkFiles.get("planet_prediction_df.csv"), sep=",", header=True, inferSchema=True)

In [13]:
# Extract
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://nasa-bucket.s3.amazonaws.com/unverified_planets_df.csv"
spark.sparkContext.addFile(url)
unverified_planets_df = spark.read.csv(SparkFiles.get("unverified_planets_df.csv"), sep=",", header=True, inferSchema=True)

In [14]:
# Extract
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://nasa-bucket.s3.amazonaws.com/verified_planets_df.csv"
spark.sparkContext.addFile(url)
verified_planets_df = spark.read.csv(SparkFiles.get("verified_planets_df.csv"), sep=",", header=True, inferSchema=True)

In [15]:
# Show DataFrame
nm_value_df.show()

+--------------+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+------+
|Kepler_OI_Name|Number_of_Planets|Planet_Radius_Earth|Equalibrium_Temp_K|Orbit_Period_Days|Star_Radius_Sun|Star_Temp_K|Star_Mass_Sun|Star_Surface_Gravity|Planet_Star_Radius_Ratio|Status|
+--------------+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+------+
|     K00752.01|                2|               2.26|             793.0|       9.48803557|          0.927|     5455.0|        0.919|               4.467|                0.022344|     1|
|     K00752.02|                2|               2.83|             443.0|       54.4183827|          0.927|     5455.0|        0.919|               4.467|                0.027954|     1|
|     K00753.01|                1|               14.6|           

In [16]:
# Show DataFrame
planet_prediction_df.show()

+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+----------+--------------+
|Number_of_Planets|Planet_Radius_Earth|Equalibrium_Temp_K|Orbit_Period_Days|Star_Radius_Sun|Star_Temp_K|Star_Mass_Sun|Star_Surface_Gravity|Planet_Star_Radius_Ratio|Prediction|Kepler_OI_Name|
+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+----------+--------------+
|                1|               14.6|             638.0|      19.89913995|          0.868|     5853.0|        0.961|               4.544|                0.154046|         0|     K00753.01|
|                1|              12.21|            1103.0|      4.959319244|          1.082|     5712.0|        0.976|               4.359|                0.103379|         0|     K00760.01|
|                1|               7.51|      

In [17]:
# Show DataFrame
unverified_planets_df.show()

+--------------+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+------+
|Kepler_OI_Name|Number_of_Planets|Planet_Radius_Earth|Equalibrium_Temp_K|Orbit_Period_Days|Star_Radius_Sun|Star_Temp_K|Star_Mass_Sun|Star_Surface_Gravity|Planet_Star_Radius_Ratio|Status|
+--------------+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+------+
|     K00753.01|                1|               14.6|             638.0|      19.89913995|          0.868|     5853.0|        0.961|               4.544|                0.154046|     2|
|     K00760.01|                1|              12.21|            1103.0|      4.959319244|          1.082|     5712.0|        0.976|               4.359|                0.103379|     2|
|     K00777.01|                1|               7.51|           

In [18]:
# Show DataFrame
verified_planets_df.show()

+--------------+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+------+
|Kepler_OI_Name|Number_of_Planets|Planet_Radius_Earth|Equalibrium_Temp_K|Orbit_Period_Days|Star_Radius_Sun|Star_Temp_K|Star_Mass_Sun|Star_Surface_Gravity|Planet_Star_Radius_Ratio|Status|
+--------------+-----------------+-------------------+------------------+-----------------+---------------+-----------+-------------+--------------------+------------------------+------+
|     K00752.01|                2|               2.26|             793.0|       9.48803557|          0.927|     5455.0|        0.919|               4.467|                0.022344|     1|
|     K00752.02|                2|               2.83|             443.0|       54.4183827|          0.927|     5455.0|        0.919|               4.467|                0.027954|     1|
|     K00754.01|                1|              33.46|           

In [19]:
# Load
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://datavizzy.ck9tx3fw5g00.us-east-1.rds.amazonaws.com:5432/NASA_Exoplanets"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write DataFrame to active_user table in RDS
nm_value_df.write.jdbc(url=jdbc_url, table='nm_value', mode=mode, properties=config)

In [ ]:
# Write DataFrame to active_user table in RDS
planet_prediction_df.write.jdbc(url=jdbc_url, table='planet_prediction', mode=mode, properties=config)

In [20]:
# Write DataFrame to active_user table in RDS
unverified_planets_df.write.jdbc(url=jdbc_url, table='unverified_planets', mode=mode, properties=config)

In [21]:
# Write DataFrame to active_user table in RDS
verified_planets_df.write.jdbc(url=jdbc_url, table='verified_planets', mode=mode, properties=config)